In [129]:
from scipy.stats import nbinom, poisson
import numpy as np


def truncPois_CDF(y, mu):
    f_zero = poisson.pmf(0, mu)
    if y > 0:
        return (poisson.cdf(y, mu) - poisson.cdf(0, mu)) / (1 - f_zero)
    else:
        return 0
    
def truncPois_logCDF(y, mu):
    f_zero = poisson.pmf(0, mu)
    if y > 0:
        return np.log((poisson.cdf(y, mu) - poisson.cdf(0, mu)) / (1 - f_zero))
    else:
        return np.log(0)

In [154]:
def log1mexp(x):
    if np.any((x < 0) & (~np.isnan(x))):
        raise ValueError("Inputs need to be non-negative!")
    return np.where(x <= np.log(2), np.log(-np.expm1(-x)), np.log1p(-np.exp(-x)))



def qpois_trunc(p, lam, lower_tail=True, log_p=False):
    # if f(0)=0 no truncation is needed
    if (1 - poisson.pmf(0, lam)) == 1:
        print('Juhu Abkürzung')
        return poisson.ppf(p, lam)
    else:
        # Convert p to array if it's a single value
        if not isinstance(p, (list, np.ndarray)):
            p = np.array([p])
        
        # Set log-probabilities (lower tail)
        n = len(p)
        if log_p:
            logp = p
        else:
            logp = np.log(p)
            
        if not lower_tail:
            logp = log1mexp(-logp)
        
        # Set output and deal with special cases (outputs NA and Inf)
        quantiles = np.full(n, np.nan)
        nna = ~np.isnan(logp)
        nlogp = logp[nna]
        if len(nlogp) == 0:
            return quantiles
        
        quantiles[nna] = np.full(len(nna), np.inf)
        if np.min(nlogp) >= 0:
            return quantiles

        # Set log-CDF vector
        lp_max = np.max(nlogp[nlogp < 0])

        # find a adequate upper limit, starting from the extreme conservative chebychev inequality
        upper = int(lam + np.sqrt(lam * np.exp(-log1mexp(-lp_max)))) #Chebychev inequality
        while np.exp(truncPois_logCDF(upper-50, lam)) > 0.999:
            upper = upper - 50

        logcdf = np.array([truncPois_logCDF(yi, lam) for yi in range(1, int(upper)+1)]) 

        # Compute output
        for i in range(n):
            if nna[i]:
                if logp[i] < 0:
                    quantiles[i] = np.sum(logcdf < logp[i]) + 1 #+1 because 0 is truncated
        
        # Return output
        if len(quantiles) == 1:
            return quantiles[0]
        else:
            return quantiles

#log.p	logical; if TRUE, probabilities p are given as log(p)
#lower.tail	logical; if TRUE (default), probabilities are P[X≤x], otherwise, P[X>x].
def qpois(p, lam, lower_tail=True, log_p=False):
    # Convert p to array if it's a single value
    if not isinstance(p, (list, np.ndarray)):
        p = np.array([p])
    
    # Set log-probabilities (lower tail)
    n = len(p)
    if log_p:
        logp = p
    else:
        logp = np.log(p)
        
    if not lower_tail:
        logp = log1mexp(-logp)
    
    # Set output and deal with special cases (outputs NA and Inf)
    quantiles = np.full(n, np.nan)
    nna = ~np.isnan(logp)
    nlogp = logp[nna]
    if len(nlogp) == 0:
        return quantiles
    
    quantiles[nna] = np.full(len(nna), np.inf)
    if np.min(nlogp) >= 0:
        return quantiles

    # Set log-CDF vector
    lp_max = np.max(nlogp[nlogp < 0])
    upper = int(lam + np.sqrt(lam * np.exp(-log1mexp(-lp_max)))) #Chebychev inequality
    logcdf = poisson.logcdf(np.arange(upper+1), lam)

    # Compute output
    for i in range(n):
        if nna[i]:
            if logp[i] < 0:
                quantiles[i] = np.sum(logcdf < logp[i])
    
    # Return output
    if len(quantiles) == 1:
        return quantiles[0]
    else:
        return quantiles

In [187]:
# calculation of quantiles
quantiles = np.arange(0.001, 0.9999, 0.001)
quantiles = [round(q, 3) for q in quantiles] # due to binary inaccuracies

lam = 0.1

a = qpois_trunc(quantiles, lam)
b = qpois(quantiles, lam)
print(len(a))
print(len(b))
c = a == b
print(len(c[c!=True]))

999
999
950


In [180]:
q = 0.5
lam = 4
print(poisson.ppf(q, lam))
print(qpois(q, lam))
print(qpois_trunc(q, lam))

4.0
4.0
4.0
